In [1]:
%cd Cardsformer

/home/s2430014/research/enhance-cardsformer-dev/Cardsformer


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# prediction modelの読み込み

In [2]:
import os

# 環境変数を設定
os.environ["PYTHONNET_RUNTIME"] = "coreclr"

# 設定した環境変数を取得
print(os.environ["PYTHONNET_RUNTIME"])

from typing import OrderedDict
from Env.Hearthstone import Hearthstone
from Env.EnvWrapper import Environment
from Model.PredictionModel import PredictionModel
from Model.ModelWrapper import Model as PolicyModel

from transformers import AutoModel, AutoTokenizer
import torch
from Algo.encoder import Encoder
import pandas as pd

import logging
import numpy as np


coreclr


In [3]:
import numpy as np

In [4]:
import torch
from torch.utils.data import DataLoader

In [5]:
!pwd

/home/s2430014/research/enhance-cardsformer-dev/Cardsformer


In [6]:
import numpy as np
from torch.utils.data import Dataset



class PredictionDataset(Dataset):

    def __init__(self, id_list, test=False, data_name='./off_line_data'):
        super(Dataset, self).__init__()
        self.data = {
            'hand_card_names': [],
            'minion_names': [],
            'weapon_names': [],
            'hand_card_scalar': [],
            'minion_scalar': [],
            'hero_scalar': [],
            'next_state_minion_scalar': [],
            'next_state_hero_scalar': [],

        }
        for id in id_list:
            print(f"=== load prediction dataset id={id}, dataname={data_name} ===")
            if test:
                data_path = './test_data' + str(id) + '.npy'
            else:
                data_path = data_name + str(id) + '.npy'
            cur_data = np.load(data_path, allow_pickle=True).item()
            for key in cur_data:
                if key == 'secret_names':
                    continue
                self.data[key] += cur_data[key]

    def __len__(self):
        return len(self.data['hand_card_names'])

    def __getitem__(self, index, id=False):
        return self.data['hand_card_names'][index], self.data['minion_names'][index], self.data['weapon_names'][index], self.data['hand_card_scalar'][index], self.data['minion_scalar'][index], self.data['hero_scalar'][index], self.data['next_state_minion_scalar'][index], self.data['next_state_hero_scalar'][index]

In [7]:
prediction_model = PredictionModel(is_train=True)
    # prediction
checkpoint_states = torch.load("./trained_models/prediction_model4715.tar", map_location="cpu")['model_state_dict']

 # unwrap the prediction model
new_state_dict = OrderedDict()
for k, v in checkpoint_states.items():
    name = k[7:]
    new_state_dict[name] = v

prediction_model.load_state_dict(new_state_dict)
prediction_model.to("cpu")
prediction_model.eval()
device = "cpu"
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
auto_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2")
encoder = Encoder(model=auto_model, tokenizer=tokenizer)
encoder.to(device)

loss_fn = torch.nn.MSELoss()

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


RuntimeError: Error(s) in loading state_dict for PredictionModel:
	size mismatch for hand_card_feat_embed.weight: copying a param with shape torch.Size([63, 23]) from checkpoint, the shape in current model is torch.Size([63, 24]).

In [ ]:
# data = PredictionDataset([0], data_name="off_line_data_vs_ai")
data = PredictionDataset([8])
data_l = DataLoader(data, batch_size=100, shuffle=False, num_workers=0)

=== load prediction dataset id=8, dataname=./off_line_data ===


In [ ]:
losses = []
for n, batch in enumerate(data_l):
    if n % 100 == 0:
        print(f"batch {n} processed")
    
    for i in range(8):
        batch[i] = batch[i].float().to(device)
    x = [batch[i] for i in range(6)] #これはなに
    y = [batch[6], batch[7]] #これはなに
    with torch.no_grad():
        pred = prediction_model(x)

    loss1 = loss_fn(y[0], pred[0])
    loss2 = loss_fn(y[1], pred[1])
    loss = loss1 + loss2
    
    losses.append(loss)
    

In [ ]:
sum(losses) / len(losses)

In [ ]:
data = PredictionDataset([0], data_name="off_line_data_vs_ai")
data_l = DataLoader(data, batch_size=100, shuffle=False, num_workers=0)

In [ ]:
losses = []

print(f"data length is {len(data_l)}")

for n, batch in enumerate(data_l):
    if n % 100 == 0:
        print(f"batch {n} processed")
    
    for i in range(8):
        batch[i] = batch[i].float().to(device)
    x = [batch[i] for i in range(6)] #これはなに
    y = [batch[6], batch[7]] #これはなに
    with torch.no_grad():
        pred = prediction_model(x)

    loss1 = loss_fn(y[0], pred[0])
    loss2 = loss_fn(y[1], pred[1])
    loss = loss1 + loss2
    
    losses.append(loss)

In [ ]:
sum(losses) / len(losses)